In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !wget https://crisisnlp.qcri.org/data/crisismmd/CrisisMMD_v2.0.tar.gz

In [3]:
# '/content/CrisisMMD_v2.0.tar.gz'

In [4]:
# !cp -r /content/CrisisMMD_v2.0 /content/drive/MyDrive/Image-Captioning-Model

In [5]:
# !unzip '/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/crisismmd_datasplit_all.zip' -d '/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0'

In [6]:
# !kaggle datasets download -d adityajn105/glove6b50d

In [7]:
# !unzip /content/drive/MyDrive/Image-Captioning-Model/glove6b50d.zip -d /content/drive/MyDrive/Image-Captioning-Model

In [8]:
#https://xiangyutang2.github.io/image-captioning/

In [9]:
# from os import listdir
from pickle import dump
from pickle import load
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from collections import Counter
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from tensorflow.keras.utils import plot_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import os
import string

In [10]:
base_model = VGG16(include_top=True)
base_model.summary()

553476096/553467096 [==============================] - 6s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [11]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
features = dict()
for dir1 in os.listdir('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image'):
    for dir2 in os.listdir('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/' + dir1):
        for file in os.listdir('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/' + dir1 + '/' + dir2):
          try:
            img_path = '/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/' + dir1 + '/' + dir2 + '/' + file
            print(img_path)
            img = load_img(img_path, target_size=(224, 224)) #size is 224,224 by default
            x = img_to_array(img) #change to np array
            x = np.expand_dims(x, axis=0) #expand to include batch dim at the beginning
            x = preprocess_input(x) #make input confirm to VGG16 input format
            fc2_features = model.predict(x)
    
            name_id = file.split('.')[0] #take the file name and use as id in dict
            features[name_id] = fc2_features
          except:
            pass

dump(features, open('features.pkl', 'wb')) #cannot use JSON because ndarray is not JSON serializable

/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923290443126902784_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923213239508488194_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923213507147026433_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923042295141806080_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923020350429937664_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923180194902441984_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923041503953539072_0.jpg
/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/data_image/hurricane_maria/25_10_2017/923305829629136896_0.jpg
/content

In [ ]:
def load_data_set_ids(filepath):
    df = pd.read_csv(filepath, sep='\t')
    
    dataset = list()
    for image_id in df['image_id']:
        if len(image_id) < 1:
            continue
            
        dataset.append(image_id)
    
    return set(dataset)

In [ ]:
training_set = load_data_set_ids('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/task_informative_text_img_train.tsv')
dev_set = load_data_set_ids('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/task_informative_text_img_dev.tsv')
test_set = load_data_set_ids('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/task_informative_text_img_test.tsv')

In [ ]:
translator = str.maketrans("", "", string.punctuation) #translation table that maps all punctuation to None
image_captions = dict()
image_captions_train = dict()
image_captions_dev = dict()
image_captions_test = dict()
image_captions_other = dict()
corpus = list() #corpus used to train tokenizer
corpus.extend(['<START>', '<END>', '<UNK>']) #add SOS and EOS to list first
max_imageCap_len = 0

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/task_informative_text_img_train.tsv', sep='\t')
df_dev = pd.read_csv('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/task_informative_text_img_dev.tsv', sep='\t')
df_test = pd.read_csv('/content/drive/MyDrive/Image-Captioning-Model/CrisisMMD_v2.0/task_informative_text_img_test.tsv', sep='\t')
df_all = df_train.append([df_dev, df_test])

In [ ]:
df_all.head()

In [ ]:
for index, row in df_all.iterrows():
    image_id = row['image_id']
    image_cap = row['tweet_text']

    image_cap = image_cap.lower() #change to lower case
    image_cap = image_cap.translate(translator) #take out punctuation using a translation table

    image_cap = image_cap.split(' ') #split string here because following two methods works on word-level best
    image_cap = [w for w in image_cap if w.isalpha()] #keep only words that are all letters
    image_cap = [w for w in image_cap if len(w)>1]
    image_cap = '<START> ' + ' '.join(image_cap) + ' <END>' #add sentence start/end; note syntax: separator.join()

    #update maximum caption length
    if len(image_cap.split()) > max_imageCap_len:
        max_imageCap_len = len(image_cap.split())
    
    #add to dictionary
    if image_id not in image_captions:
        image_captions[image_id] = list() #creat a new list if it does not yet exist
    image_captions[image_id].append(image_cap)
    
    #add to train/dev/test dictionaries
    if image_id in training_set:
        if image_id not in image_captions_train:
            image_captions_train[image_id] = list() #creat a new list if it does not yet exist
        image_captions_train[image_id].append(image_cap)
        corpus.extend(image_cap.split()) #add only training words to corpus to train tokenlizer
        
    elif image_id in dev_set:
        if image_id not in image_captions_dev:
            image_captions_dev[image_id] = list() #creat a new list if it does not yet exist
        image_captions_dev[image_id].append(image_cap)
        
    elif image_id in test_set:
        if image_id not in image_captions_test:
            image_captions_test[image_id] = list() #creat a new list if it does not yet exist
        image_captions_test[image_id].append(image_cap)
    else:
        if image_id not in image_captions_other:
            image_captions_other[image_id] = list() #creat a new list if it does not yet exist
        image_captions_other[image_id].append(image_cap)

caption_train_tokenizer = Tokenizer() #initialize tokenizer
caption_train_tokenizer.fit_on_texts(corpus) #fit tokenizer on training data
    
fid = open("image_captions.pkl","wb")
dump(image_captions, fid)
fid.close()

fid = open("image_captions_train.pkl","wb")
dump(image_captions_train, fid)
fid.close()

fid = open("image_captions_dev.pkl","wb")
dump(image_captions_dev, fid)
fid.close()

fid = open("image_captions_test.pkl","wb")
dump(image_captions_test, fid)
fid.close()

fid = open("image_captions_other.pkl","wb")
dump(image_captions_other, fid)
fid.close()

fid = open("caption_train_tokenizer.pkl","wb")
dump(caption_train_tokenizer, fid)
fid.close()

fid = open("corpus.pkl","wb")
dump(corpus, fid)
fid.close()

corpus_count=Counter(corpus)
fid = open("corpus_count.pkl","wb")
dump(corpus_count, fid)
fid.close()

print("size of data =", len(image_captions), "size of training data =", len(image_captions_train), "size of dev data =", len(image_captions_dev), "size of test data =", len(image_captions_test), "size of unused data =", len(image_captions_other))
print("maximum image caption length =",max_imageCap_len)

In [ ]:
!kaggle datasets download -d adityajn105/glove6b50d

In [ ]:
embeddings_index = dict()
fid = open('/content/drive/MyDrive/Image-Captioning-Model/glove.6B.50d.txt' ,encoding="utf8")
for line in fid:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
fid.close()

In [ ]:
EMBEDDING_DIM = 50
word_index = caption_train_tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

for word, idx in word_index.items():
    embed_vector = embeddings_index.get(word)
    if embed_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[idx] = embed_vector
        
fid = open("embedding_matrix.pkl","wb")
dump(embedding_matrix, fid)
fid.close()

In [ ]:
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0] #[0] is used to take out the extra dim. This changes from text to a number
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            # import pdb; pdb.set_trace()
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(np.squeeze(X1)), np.array(X2), np.array(y)

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length, batch_size, vocab_size):
    # loop for ever over images
    current_batch_size=0
    while 1:
        for key, desc_list in descriptions.items():
            # retrieve the photo feature
            if current_batch_size == 0:
                X1, X2, Y = list(), list(), list()
            
            imageFeature_id = key.split('.')[0]
            photo = photos[imageFeature_id][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
            #in_img = np.squeeze(in_img)
            X1.extend(in_img)
            X2.extend(in_seq)
            Y.extend(out_word)
            current_batch_size += 1
            if current_batch_size == batch_size:
                current_batch_size = 0
                yield [np.array(X1), np.array(X2)], np.array(Y)

In [ ]:
from pickle import load
fid = open('features.pkl', 'rb')
image_features = load(fid)
fid.close()

In [ ]:
# test the data generator
caption_max_length = max_imageCap_len
batch_size = 1
vocab_size = len(caption_train_tokenizer.word_index)
generator = data_generator(image_captions_train, image_features, caption_train_tokenizer, caption_max_length, batch_size, vocab_size)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

In [ ]:
from keras.layers.merge import concatenate
def define_model_concat(vocab_size, max_length, embedding_matrix):
    # feature extractor model
    inputs1 = Input(shape=(4096,))
    image_feature = Dropout(0.5)(inputs1)
    image_feature = Dense(256, activation='relu')(image_feature)
    # sequence model
    inputs2 = Input(shape=(max_length,))
    language_feature = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=max_length, trainable=False)(inputs2)
    #Embedding(vocab_size, 256, mask_zero=True)(inputs2) #<<<<<< fix me, add pretrianed embedding
    language_feature = Dropout(0.5)(language_feature)
    language_feature = LSTM(256)(language_feature)
    # decoder model
    output = concatenate([image_feature, language_feature])
    output = Dense(256, activation='relu')(output)
    output = Dense(vocab_size, activation='softmax')(output)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model_concat.png', show_shapes=True)
    return model

fid = open("embedding_matrix.pkl","rb")
embedding_matrix = load(fid)
fid.close()

caption_max_length = max_imageCap_len
vocab_size = len(caption_train_tokenizer.word_index)
post_rnn_model_concat = define_model_concat(vocab_size, caption_max_length, embedding_matrix)

In [ ]:
fid = open("features.pkl","rb")
image_features = load(fid)
fid.close()

fid = open("caption_train_tokenizer.pkl","rb")
caption_train_tokenizer = load(fid)
fid.close()

fid = open("image_captions_train.pkl","rb")
image_captions_train = load(fid)
fid.close()

fid = open("image_captions_dev.pkl","rb")
image_captions_dev = load(fid)
fid.close()

caption_max_length = max_imageCap_len
batch_size = 100
vocab_size = len(caption_train_tokenizer.word_index)
#generator = data_generator(image_captions_train, image_features, caption_train_tokenizer, caption_max_length, batch_size, vocab_size)

#epochs = 2
#steps = len(image_captions_train)
#steps_per_epoch = np.floor(steps/batch_size)

In [ ]:
batch_size = 6
steps = len(image_captions_train)
steps_per_epoch = np.floor(steps/batch_size)

epochs = 100

for i in range(epochs):
	# create the data generator
	generator = data_generator(image_captions_train, image_features, caption_train_tokenizer, caption_max_length, batch_size, vocab_size)
	# fit for one epoch
	post_rnn_model_concat_hist=post_rnn_model_concat.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
	# save model
	post_rnn_model_concat.save('modelConcat_1_' + str(i) + '.h5')

In [ ]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model

In [ ]:
base_model = VGG16(include_top=True)
feature_extract_pred_model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [ ]:
def extract_feature(model, file_name):
    img = load_img(file_name, target_size=(224, 224)) #size is 224,224 by default
    x = img_to_array(img) #change to np array
    x = np.expand_dims(x, axis=0) #expand to include batch dim at the beginning
    x = preprocess_input(x) #make input confirm to VGG16 input format
    fc2_features = model.predict(x)
    return fc2_features

In [ ]:
# load the tokenizer
caption_train_tokenizer = load(open('caption_train_tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = max_imageCap_len
# load the model
#pred_model = load_model('model_3_0.h5')
pred_model = load_model('modelConcat_1_40.h5')

In [ ]:
def generate_caption(pred_model, caption_train_tokenizer, photo, max_length):
    in_text = '<START>'
    caption_text = list()
    for i in range(max_length):
            # integer encode input sequence
            sequence = caption_train_tokenizer.texts_to_sequences([in_text])[0]
            # pad input
            sequence = pad_sequences([sequence], maxlen=max_length)
            # predict next word
            model_softMax_output = pred_model.predict([photo,sequence], verbose=0)
            # convert probability to integer
            word_index = argmax(model_softMax_output)
            # map integer to word
            word = caption_train_tokenizer.index_word[word_index]
            #print(word)
            # stop if we cannot map the word
            if word is None:
                break
            # append as input for generating the next word
            in_text += ' ' + word
            # stop if we predict the end of the sequence
            if word != 'end':
                caption_text.append(word)
            if word == 'end':
                break
    return caption_text

In [ ]:
import numpy as np
caption_image_fileName = '/home/rangika/PythonCode/PHASE3/PROJECT/Multimodal/images/news/LM74FMR62Y7ACUXN3JYDB3OJOM.jpg'
photo = extract_feature(feature_extract_pred_model, caption_image_fileName)
caption = generate_caption(pred_model, caption_train_tokenizer, photo, max_length)
print(' '.join(caption))